https://github.com/LunaBlack/KGAT-pytorch

In [1]:
import os
import pandas as pd
import numpy as np
import scipy.sparse as sp
import argparse

from sklearn.preprocessing import LabelEncoder

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

In [2]:
path = '../Data/Amazon-Book'

In [62]:
entity_list_df = pd.read_csv(os.path.join(path, 'entity_list.txt'), names = ['org_id', 'remap_id'], sep = '\s', error_bad_lines = False, header = 0)

/tmp/ipykernel_6097/3509060418.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  entity_list_df = pd.read_csv(os.path.join(path, 'entity_list.txt'), names = ['org_id', 'remap_id'], sep = '\s', error_bad_lines = False, header = 0)
/tmp/ipykernel_6097/3509060418.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  entity_list_df = pd.read_csv(os.path.join(path, 'entity_list.txt'), names = ['org_id', 'remap_id'], sep = '\s', error_bad_lines = False, header = 0)
Skipping line 25710: Expected 2 fields in line 25710, saw 4. Error could possibly be due to quotes being ignored when a multi-char delimiter is used.
Skipping line 40039: Expected 2 fields in line 40039, saw 3. Error could possibly be due to quot

In [63]:
entity_list_df.head()

,org_id,remap_id
0,m.045wq1q,0.0
1,m.03_28m,1.0
2,m.0h2q1cq,2.0
3,m.04y9jxd,3.0
4,m.060c1r,4.0


In [64]:
item_list_df = pd.read_csv(os.path.join(path, 'item_list.txt'), names = ['org_id', 'remap_id', 'freebase_id'], sep = '\s', header = 0)

/tmp/ipykernel_6097/1822623027.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  item_list_df = pd.read_csv(os.path.join(path, 'item_list.txt'), names = ['org_id', 'remap_id', 'freebase_id'], sep = '\s', header = 0)


In [65]:
item_list_df.head()

,org_id,remap_id,freebase_id
0,0553092626,0,m.045wq1q
1,0393316041,1,m.03_28m
2,038548254X,2,m.0h2q1cq
3,0385307756,3,m.04y9jxd
4,038531258X,4,m.060c1r


In [78]:
set(item_list_df.groupby(['org_id'])['freebase_id'].nunique())

{1}

In [79]:
set(item_list_df.groupby(['freebase_id'])['org_id'].nunique())

{1}

In [80]:
item_list_df['org_id'].nunique() - item_list_df['freebase_id'].nunique()

0

In [53]:
kg_final_df = pd.read_csv(os.path.join(path, 'kg_final.txt'), names = ['h', 'r', 't'], sep = ' ')

In [54]:
kg_final_df.head()

,h,r,t
0,24915,0,24916
1,24917,1,5117
2,24918,0,24917
3,24919,1,24920
4,24921,2,24922


In [59]:
user_list_df = pd.read_csv(os.path.join(path, 'user_list.txt'), names = ['org_id', 'remap_id'], sep = '\s', header = 0)

/tmp/ipykernel_6097/2992960167.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  user_list_df = pd.read_csv(os.path.join(path, 'user_list.txt'), names = ['org_id', 'remap_id'], sep = '\s', header = 0)


In [60]:
user_list_df.head()

,org_id,remap_id
0,A3RTKL9KB8KLID,0
1,A38LAIK2N83NH0,1
2,A3PPXVR5J6U2JD,2
3,A2ULDDL3MLJPUR,3
4,A2I6MHMAZZDCRX,4


In [81]:
user_list_df['remap_id'].nunique()

70679

In [82]:
item_list_df['remap_id'].nunique()

24915

In [83]:
kg_final_df['h'].nunique()

113308

In [84]:
kg_final_df['t'].nunique()

113479

In [85]:
entity_list_df['remap_id'].nunique()

113472

In [88]:
kg_final_df['t'].nunique() - (user_list_df['remap_id'].nunique() + item_list_df['remap_id'].nunique())

17885

In [89]:
kg_final_df['t'].nunique()

113479

In [90]:
kg_final_df['h'].nunique()

113308

In [91]:
kg_final_df.groupby(['t'])['h'].nunique()

t
0         16
1         10
2          6
3         10
4         15
          ..
113482     5
113483     5
113484     5
113485     6
113486     6
Name: h, Length: 113479, dtype: int64